In [1]:
import os
import numpy as np
from osgeo import gdal

In [ ]:
## Background. 

This notebook has been created for G20 DRR Hackathon for Team MapleByte (Canada). This works towards the 'Settlement Detection' portion of the challenge.  
Project/team site: [g20hack-maplebyte.climatechange.ca](https://g20hack-maplebyte.climatechange.ca/)

## Binary Maps of Settlements
- The binary maps of settlements were generated using Sentinel-2 satellite imagery available on the Digital Earth Africa (DEA) platform and the Urban Change Detection script, also available on the DEA.
- The process involved several key steps:
1. **AOI selection**:
A GeoJSON file delineating the area of interest (AOI) was used to focus the analysis on a specific geographic region, specifically, Durban, South Africa.
2. **Data Acquisition**: Sentinel-2 imagery was accessed through the DEA platform. For the analysis, we acquired images from 2019 to 2025, with a focus on 2022 when a major flood event occurred from 8 – 18 April 2022.
For the pre-flood period, the image from 29 March 2022 was used, while for the post-flood period, the image from 28 April 2022 was utilized, as these were the closest cloud-free images available around the event.
We used the swir_1, swir_2, blue, green, and red bands and set a spatial resolution of 10 meters.
For the years 2019–2021 and 2023–2024, we used annual geomedians; for 2025, we used the image from 8 March 2025, because a geomedian was not available for that year.
3. **Preprocessing**: The acquired images were used to calculate the Enhanced Normalized Difference Impervious Surfaces Index (ENDISI), and Otsu thresholding was applied to classify pixels into urban and non-urban areas.
4. **Post-processing**: Because large amounts of sediment were deposited in the river after the flood event, the river was misidentified as a settlement area. To correct this misclassification, we masked out the river using a water mask derived from the ESRI annual classification map for 2022. We applied corresponding water masks for other years as well to ensure consistency across the time series.
As ESRI doesn’t have a classification map for 2024 and 2025, we used the 2023 water mask for those years.

In [8]:
IMAGE_DATA = "refined_binary_maps/"

In [9]:
f_list = os.listdir(IMAGE_DATA)
f_list.sort()
pairs_list = [(f_list[i], f_list[i+1]) for i in range(len(f_list)-1)]

for pair in pairs_list:
    img1_ds = gdal.Open(os.path.join(IMAGE_DATA, pair[0]), 0)
    img1_arr = img1_ds.ReadAsArray()

    img2_ds = gdal.Open(os.path.join(IMAGE_DATA, pair[1]), 0)
    img2_arr = img2_ds.ReadAsArray()

    change_map = img1_arr - img2_arr

    change_map = np.where(change_map < 0, 1, np.where(change_map > 0, -1, 0))

    before_prefix = pair[0].replace("_binary.tif", "")
    after_prefix = pair[1].replace("_binary.tif", "")
    out_file_name = f"{before_prefix}_to_{after_prefix}_change_map.tif"
    out_folder = os.path.join(os.path.dirname(os.path.abspath(__file__)), "change_maps")
    os.makedirs(out_folder, exist_ok=True)
    out_file = os.path.join(out_folder, out_file_name)

    drv = gdal.GetDriverByName('GTiff')
    out_ds = drv.Create(out_file, change_map.shape[1], change_map.shape[0], 1, gdal.GDT_Int16, options=['COMPRESS=DEFLATE'])
    out_ds.SetGeoTransform(img1_ds.GetGeoTransform())
    out_ds.SetProjection(img1_ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(change_map)
    out_ds.GetRasterBand(1).SetNoDataValue(0)
    out_ds.FlushCache()
    out_ds = None
    img1_ds = None
    img2_ds = None

FileNotFoundError: [Errno 2] No such file or directory: 'refined_binary_maps/'

In [ ]:
## Change maps of settlements
================================
- To generate change maps of settlements, we used the binary settlement maps created in the previous step. Each map was compared to the map from the previous year to identify changes in settlement areas. The changes were categorized into three classes:
    - No Change (0): Areas where there was no change in settlement status between the two years.
    - Settlement Gain (1): Areas that were classified as non-settlement in the previous year but were classified as settlement in the current year.
    - Settlement Loss (−1): Areas that were classified as settlement in the previous year but were classified as non-settlement in the current year.

In [ ]:
MAP = {
    "2019_binary.tif": "2019_ESRI.tif",
    "2020_binary.tif": "2020_ESRI.tif",
    "2021_binary.tif": "2021_ESRI.tif",
    "2022_0_before_binary.tif": "2022_ESRI.tif",
    "2022_1_after_binary.tif": "2022_ESRI.tif",
    "2023_binary.tif": "2023_ESRI.tif",
    "2024_binary.tif": "2023_ESRI.tif",
    "2025_binary.tif": "2023_ESRI.tif",
}
BINARY_DATA = "binary_maps/"
LULC_MAPS = "lulc_maps/"
OUTPUT_DIR = "refined_binary_maps/"

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
    for binary_map, lulc_map in MAP.items():
        binary_path = os.path.join(BINARY_DATA, binary_map)
        lulc_path = os.path.join(LULC_MAPS, lulc_map)

        binary_ds = gdal.Open(binary_path, gdal.GA_ReadOnly)
        binary_arr = binary_ds.ReadAsArray()

        lulc_ds = gdal.Open(lulc_path, gdal.GA_ReadOnly)
        lulc_arr = lulc_ds.ReadAsArray()

        # Refine binary map: keep only urban areas
        refined_binary_arr = np.where(lulc_arr == 1, 0, binary_arr)

        out_file = os.path.join(OUTPUT_DIR, binary_map.replace("_binary.tif", "_refined_binary.tif"))

        drv = gdal.GetDriverByName('GTiff')
        out_ds = drv.Create(out_file, refined_binary_arr.shape[1], refined_binary_arr.shape[0], 1, gdal.GDT_Byte, options=['COMPRESS=DEFLATE'])
        out_ds.SetGeoTransform(binary_ds.GetGeoTransform())
        out_ds.SetProjection(binary_ds.GetProjection())
        out_ds.GetRasterBand(1).WriteArray(refined_binary_arr)
        out_ds.GetRasterBand(1).SetNoDataValue(0)
        out_ds.FlushCache()
        out_ds = None
        binary_ds = None
        lulc_ds = None